In [1]:
# Libraries for data manipulation and linear algebra
import numpy as np 
import pandas as pd 

# For file paths
from pathlib import Path  

# Tensorflow and Keras for Neural Network
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

# Scikit-learn libraries for preprocessing, model evaluation, and text vectorization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import mean_squared_error
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel
import torch

# Libraries for text processing
import re
import spacy
from spacy.lang.en import English 
from spacy.lang.en.stop_words import STOP_WORDS

# Visualization
import matplotlib.pyplot as plt

%matplotlib inline

# Display input files
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/huggingface-bertmodel-base-uncased/tokenizer/tokenizer_config.json
/kaggle/input/huggingface-bertmodel-base-uncased/tokenizer/special_tokens_map.json
/kaggle/input/huggingface-bertmodel-base-uncased/tokenizer/vocab.txt
/kaggle/input/huggingface-bertmodel-base-uncased/Bert-base-uncased/config.json
/kaggle/input/huggingface-bertmodel-base-uncased/Bert-base-uncased/pytorch_model.bin
/kaggle/input/summary-test/summaries_test.csv
/kaggle/input/bertmodel/bert.py
/kaggle/input/bertmodel/config.json
/kaggle/input/bertmodel/special_tokens_map.json
/kaggle/input/bertmodel/vocab.txt
/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv


Load and Preprocess Dataset

In [2]:
# Load the dataset 

file_path = '/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv'
se = pd.read_csv(file_path)

# Drop rows with missing 'wording' and 'content'
se.dropna(subset=['wording', 'content'], inplace=True)

# Fill missing 'text' values
se['text'].fillna('', inplace=True)

# Text preprocessing function
def preprocess_text(text):
    text = text.replace('\n', ' ').lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = ' '.join([word for word in text.split() if word.isalpha() and word not in STOP_WORDS])
    return ' '.join(text.split())

# Apply the preprocessing function to texts
se['PreprocessedText'] = se['text'].apply(preprocess_text)


Vectorization of Text Data

In [3]:
# Vectorize preprocessed texts
MODEL_DIR = "/kaggle/input/huggingface-bertmodel-base-uncased/"
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR + "tokenizer")
bert_model = AutoModel.from_pretrained(MODEL_DIR + "Bert-base-uncased")

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        output = bert_model(**inputs)
    # The model's output is a tuple, where the first element is the embeddings
    embeddings = output[0]
    # Taking the mean of the word embeddings as representation
    return embeddings.mean(dim=1).numpy()

X = np.array([get_bert_embedding(text) for text in se['PreprocessedText']])

# Split data into training and testing sets
y = se[['wording', 'content']].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Data Preparation for Neural Network

In [4]:
# Reshape data into 2D array
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)


# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Neural Network Model Building and Training

In [5]:
# Define model architecture
hidden_nodes_l1 = (X_train_scaled.shape[1] + 1) // 2
hidden_nodes_l2 = (hidden_nodes_l1 + 1) // 2
hidden_nodes_l3 = (hidden_nodes_l2 + 1) // 2
hidden_nodes_l4 = (hidden_nodes_l3 + 1) // 2

model = Sequential([
    Dense(hidden_nodes_l1, input_dim=X_train_scaled.shape[1], activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.25),
    Dense(hidden_nodes_l2, activation='tanh', kernel_regularizer=l2(0.001)),
    Dropout(0.25),
    Dense(hidden_nodes_l3, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.25),
    Dense(hidden_nodes_l4, activation='tanh', kernel_regularizer=l2(0.001)),
    # Output layer with two nodes for "wording" and "content"
    Dense(2, activation='sigmoid')  
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
144/144 [==============================] - 3s 9ms/step - loss: 1.6454 - mse: 0.8365 - val_loss: 1.3605 - val_mse: 0.7279
Epoch 2/50
144/144 [==============================] - 1s 7ms/step - loss: 1.2778 - mse: 0.7819 - val_loss: 1.1430 - val_mse: 0.7614
Epoch 3/50
144/144 [==============================] - 1s 7ms/step - loss: 1.0811 - mse: 0.7751 - val_loss: 0.9801 - val_mse: 0.7397
Epoch 4/50
144/144 [==============================] - 1s 7ms/step - loss: 0.9630 - mse: 0.7643 - val_loss: 0.8843 - val_mse: 0.7218
Epoch 5/50
144/144 [==============================] - 1s 8ms/step - loss: 0.8923 - mse: 0.7551 - val_loss: 0.8340 - val_mse: 0.7196
Epoch 6/50
144/144 [==============================] - 1s 8ms/step - loss: 0.8593 - mse: 0.7575 - val_loss: 0.8204 - val_mse: 0.7298
Epoch 7/50
144/144 [==============================] - 1s 7ms/step - loss: 0.8365 - mse: 0.7548 - val_loss: 0.8018 - val_mse: 0.7292
Epoch 8/50
144/144 [==============================] - 1s 7ms/step - loss: 0.

Evaluation

In [6]:
# Predict on test data
y_pred = model.predict(X_test_scaled)

rmse_wording = mean_squared_error(y_test[:, 0], y_pred[:, 0], squared=False)
rmse_content = mean_squared_error(y_test[:, 1], y_pred[:, 1], squared=False)

print(f"RMSE for wording: {rmse_wording}")
print(f"RMSE for content: {rmse_content}")


45/45 [==============================] - 0s 2ms/step
RMSE for wording: 0.8931649571177654
RMSE for content: 0.8336680928919784


In [7]:
# Load test dataset for competition
test_df = pd.read_csv(Path("/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv"))

# Preprocess and vectorize the test data
test_df['PreprocessedText'] = test_df['text'].apply(preprocess_text)
X_test_competition = np.array([get_bert_embedding(text) for text in test_df['PreprocessedText']])

X_test_competition = X_test_competition.reshape(X_test_competition.shape[0], -1)
print(X_test_competition.shape)



(4, 768)


Prepare Submission for Competition

In [8]:
# Predict using the trained model
predictions = model.predict(X_test_competition)

print(predictions.shape)

1/1 [==============================] - 0s 21ms/step
(4, 2)


In [9]:
# Flatten predictions
predictions = predictions.reshape(-1, 2)
print(predictions.shape)

(4, 2)


In [10]:
# Prepare submission DataFrame
submission_df = pd.DataFrame({
    'student_id': test_df['student_id'],
    'content': predictions[:, 1],
    'wording': predictions[:, 0]
})

# Clip values to be within expected bounds
submission_df['content'] = np.clip(submission_df['content'], -2, 5)
submission_df['wording'] = np.clip(submission_df['wording'], -2, 5)

# Save submission to a CSV
submission_df.to_csv("submission.csv", index=False)